# [Homework 3](https://github.com/Sapienza-University-Rome/ADM/tree/master/2024/Homework_3) - **Michelin restaurants in Italy**
![iStock-654454404-777x518](https://a.storyblok.com/f/125576/2448x1220/327bb24d32/hero_update_michelin.jpg/m/1224x0/filters:format(webp))

## 1. **Data collection**

For the data collection, we wrote the required function in a `data_collection.py` module. 

In [34]:
from data_collection import save_links, download_html_from_link_file, html_to_tsv

The following is the overview of the main functions for each step, together with the code to run. 

Every function has an optional `data_folder` argument wich server the purpose to set the working data directory. 
We tought this to be useful, for example to set the date of the data collection as the directory name. 
This is useful, as the Michelin list of restaurant is constantly updated. 

In [35]:
data_folder = 'DATA'
# date of last data collection, yy-mm-dd

---

### 1.1. Get the list of Michelin restaurants
   #### **Function**: `save_links`
   - **Description**: 
     Collects restaurant links from the Michelin Guide website starting from the provided `start_url`. The links are saved into a text file (`restaurant_links.txt`) within a specified data folder.
   - **Input**: 
     - `start_url`: URL of the Michelin Guide page to start scraping.
   - **Optional Input**: 
     - `file_name`: name of the output file; by default it is `restaurant_links.txt`.
     - `data_folder`: the folder where datas will be stored; by default it is `DATA`.
   - **Output**:
     - A text file containing restaurant links, one per line, saved in the `data_folder`.
   - **Key Features**:
     - Automatically detects the number of pages to scrape.
     - Skips scraping if the links file already exists.

In [36]:
start_url = "https://guide.michelin.com/en/it/restaurants"
save_links(start_url, data_folder = data_folder)

Links already collected.
There are 1981 link already collected


---

### 1.2. Crawl Michelin restaurant pages
   #### **Function**: `download_html_from_link_file`
   - **Description**: 
     Downloads the HTML from every URL in the input `file_name`, and saves them to a structured folder (`DATA/HTMLs/page_X`).
   - **Input (all optional)**:
     - `file_name`: name of the file with the links; by default it is `restaurant_links.txt`.
     - `data_folder`: the folder where datas will be stored; by default it is `DATA`.
   - **Output**:
     - Saves the HTML files in a structured folder `DATA/HTMLs/page_X`. 
   - **Key Features**:
     - Uses `ThreadPoolExecutor` to speed up the process
     - Skips existing HTML files

In [37]:
download_html_from_link_file(data_folder = data_folder)

Download HTMLs: 100%|██████████| 1981/1981 [00:00<00:00, 34544.77it/s]

All html files have been saved.


---

### 1.3 Parse downloaded pages

#### **Function**: `extract_info_from_html`
- **Description**:  
  Parses a restaurant's HTML page and extracts structured information such as name, address, cuisine type, price range, description, and services.
- **Input**:
  - `html`: The raw HTML content of a restaurant's page.
- **Output**:
  - A dictionary containing extracted fields.
- **Key Features**:
  - Handles missing data gracefully.
  - Handles addresses separated by commas.


#### **Function**: `html_to_tsv`
- **Description**:  
  Scans the `HTMLs` folder inside the `data_folder` for all the html files, then processes every file with `extract_info_from_html`.
- **Input (optional)**:
  - `data_folder`: The folder where data will be stored; by default it is `DATA`.
  - `max_workers`: the max number of concurrent HTML parsing tasks. 
- **Output**:
  - Saves the TSV files in the folder `DATA/TSVs`.
- **Key Features**:
     - Uses `ThreadPoolExecutor` to speed up the process. 
- **Advice**:
     - Fine-tune the `max_workers` parameter according to your CPU performance. As a rule of thumb, set `max_workers` to the number of CPU cores available. An estimated processing time of around 5 minutes is typical. 

In [38]:
html_to_tsv(data_folder=data_folder, max_workers=4)

Processing HTMLs: 100%|██████████| 1981/1981 [00:00<00:00, 123067.71it/s]

All files have been processed and saved.


For completeness, let us create the dataframe for our dataset, in order to handle it effectively.

#### **Function**: `create_combined_dataframe`
- **Description**:  
  This function reads all the `.tsv` files from a specified folder, loads them into individual pandas DataFrames, and then combines them into a single DataFrame. It is useful for aggregating data from multiple sources into one unified dataset for further analysis.

- **Input**:
  - `folder_path` (str): The path to the folder containing the `.tsv` files to be read.
  - `separator` (str): The delimiter used in the `.tsv` files. Typically, it's a tab (`\t`), but it could be adjusted if needed.
  
- **Output**:
  - Returns a pandas DataFrame containing all the combined data from the `.tsv` files in the specified folder.

- **Key Features**:
  - Utilizes `glob` to find all `.tsv` files in the provided folder.
  - Loads each file as a DataFrame using pandas `read_csv()` with the specified delimiter.
  - Concatenates all DataFrames into one, ignoring index to prevent duplication.
  - Efficient handling of large datasets through pandas' built-in functions.

By running this function, you'll have a consolidated view of all the restaurant data in a single DataFrame, ready for any further analysis or processing. The first few rows of the dataset are provided below.

In [39]:
import os
from data_collection import create_combined_dataframe
df = create_combined_dataframe(r'DATA/TSVs', separator='\t')
df.head()

,restaurantName,address,city,postalCode,region,country,latitude,longitude,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,L'Acciuga,via Settevalli 217,Perugia,6128,Umbria,Italy,43.090152,12.368430,€€€,"Contemporary, International",You would never guess that there was a gourmet...,"[Air conditioning, Interesting wine list, Terr...","[amex, unionpay, dinersclub, discover, jcb, ma...",+39 339 263 2591,https://www.lacciuga.net/
1,Jamantè,via San Vito 97,Polignano a Mare,70044,Apulia,Italy,40.997814,17.212649,€€€,Modern Cuisine,Located not far from the historic centre and t...,"[Air conditioning, Terrace]","[amex, dinersclub, mastercard, visa]",+39 351 628 7773,https://www.jamanteristorante.com
2,Cucine Nervi,corso Vercelli 117,Gattinara,13045,Piedmont,Italy,45.610780,8.372140,€€€,Modern Cuisine,Housed within the Cantine Nervi in the town ce...,[Air conditioning],"[mastercard, visa]",+39 333 182 4123,https://cucinenervi.com
3,Lokanda Devetak,via Brezici 22,Savogna d'Isonzo,34070,Friuli-Venezia Giulia,Italy,45.883552,13.560400,€€,"Regional Cuisine, Traditional Cuisine",Just two kilometres from the Slovenian border ...,"[Air conditioning, Car park, Garden or park, I...","[amex, mastercard, visa]",+39 0481 882488,https://www.devetak.com/
4,Felix Lo Basso home & restaurant,via Carlo Goldoni 36,Milan,20129,Lombardy,Italy,45.469198,9.213598,€€€€,"Italian Contemporary, Creative",Brilliant chef Felix Lo Basso’s menu is inspir...,"[Air conditioning, Counter dining, Wheelchair ...","[amex, mastercard, visa]",+39 02 4540 9759,https://www.felixlobassorestaurant.it/


---

## 2. **Search Engine**

In this section, we developed two types of search engines: a **Conjunctive Search Engine** and a **Ranked Search Engine**. These engines allow users to retrieve restaurant information by processing their queries effectively. Our focus here is specifically on queries related to restaurants' descriptions.

### 2.0 Preprocessing the Text

First, we will clean and prepare the restaurant descriptions data using the `nltk` library. Let's start by installing and downloading the necessary library and packages.

In [40]:
!pip install --upgrade nltk

In [41]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

Let us now add a new column to the DataFrame named `processedDescription`. This column will store the processed versions of the restaurant descriptions, refined by removing stopwords, cleaning punctuation, and applying stemming.

#### **Function**: `preprocess_text`
- **Description:**  
    This function preprocesses a list of restaurant descriptions to enhance their suitability for search and retrieval tasks. The function performs several preprocessing steps including tokenization, removal of common stopwords, punctuation cleaning, and word stemming. These operations help streamline search processes by reducing descriptions to their core, searchable components.

- **Input**
    - `text` (str): A string representing the text input.

- **Output**
    - `processed_text` (list of list of str): A list in which each element is a list of processed tokens corresponding to each word in the input text. Each token is a cleaned, stemmed version of the original words in the provided text.

- **Key Features**
    - **Tokenization**: Divides each text into individual words or punctuation marks for further processing.
    - **Stopword Removal**: Filters out commonly used words that are less meaningful for search and classification.
    - **Punctuation Cleaning**: Removes non-alphanumeric characters to focus on the essential content.
    - **Stemming**: Reduces each word to its root form, facilitating matches across different morphological variants.

In [42]:
# Import the preprocess_text function from the search_engine module
from search_engine import preprocess_text

# Apply the preprocess_text function to the 'description' column in the DataFrame
# and store the results in a new column named 'processedDescription'
df['processedDescription'] = preprocess_text(df['description'])

# Display the first few rows of the DataFrame to verify the new column
df.head()

,restaurantName,address,city,postalCode,region,country,latitude,longitude,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,processedDescription
0,L'Acciuga,via Settevalli 217,Perugia,6128,Umbria,Italy,43.090152,12.368430,€€€,"Contemporary, International",You would never guess that there was a gourmet...,"[Air conditioning, Interesting wine list, Terr...","[amex, unionpay, dinersclub, discover, jcb, ma...",+39 339 263 2591,https://www.lacciuga.net/,"[would, never, guess, gourmet, restaur, tuck, ..."
1,Jamantè,via San Vito 97,Polignano a Mare,70044,Apulia,Italy,40.997814,17.212649,€€€,Modern Cuisine,Located not far from the historic centre and t...,"[Air conditioning, Terrace]","[amex, dinersclub, mastercard, visa]",+39 351 628 7773,https://www.jamanteristorante.com,"[locat, far, histor, centr, picturesqu, lama, ..."
2,Cucine Nervi,corso Vercelli 117,Gattinara,13045,Piedmont,Italy,45.610780,8.372140,€€€,Modern Cuisine,Housed within the Cantine Nervi in the town ce...,[Air conditioning],"[mastercard, visa]",+39 333 182 4123,https://cucinenervi.com,"[hous, within, cantin, nervi, town, centr, con..."
3,Lokanda Devetak,via Brezici 22,Savogna d'Isonzo,34070,Friuli-Venezia Giulia,Italy,45.883552,13.560400,€€,"Regional Cuisine, Traditional Cuisine",Just two kilometres from the Slovenian border ...,"[Air conditioning, Car park, Garden or park, I...","[amex, mastercard, visa]",+39 0481 882488,https://www.devetak.com/,"[two, kilometr, slovenian, border, crow, fli, ..."
4,Felix Lo Basso home & restaurant,via Carlo Goldoni 36,Milan,20129,Lombardy,Italy,45.469198,9.213598,€€€€,"Italian Contemporary, Creative",Brilliant chef Felix Lo Basso’s menu is inspir...,"[Air conditioning, Counter dining, Wheelchair ...","[amex, mastercard, visa]",+39 02 4540 9759,https://www.felixlobassorestaurant.it/,"[brilliant, chef, felix, lo, basso, menu, insp..."


### 2.1 Conjunctive Query

Next, we will construct the **Conjunctive Search Engine**, which retrieves restaurants whose descriptions contain all specified query terms.

#### 2.1.1 Create Your Index!

In this section, we create or load a file named `"vocabulary.csv"`, which maps each unique word to a corresponding integer identifier (`term_id`). We assign integer values from $0$ up to the total number of unique words in the processed descriptions minus one. 

To optimize computation and avoid redundant processing, if the file already exists, we simply load it. For efficient usage, the data is stored in a DataFrame called `vocabulary_df`.

#### **Function**: `get_vocabulary`

- **Description:**  
    This function either loads an existing `"vocabulary.csv"` file or creates a new one if it does not exist. The vocabulary file maps each unique word (or "term") found in the processed text descriptions to a unique integer ID, which can be used to reference the term efficiently.

- **Input**
    - `processed_texts` (list of list of str): A list where each sublist contains tokenized and processed words from a text.
    - `file_path` (str, default=`"vocabulary.csv"`): Path to the `.csv` file where the vocabulary will be stored or loaded from.

- **Output**
    - `vocabulary_df` (`pd.DataFrame`): A DataFrame that contains two columns: `term_id`, the unique integer ID assigned to each term, and `term`, the corresponding word.

- **Key Features**
    - **File Existence Check**: Checks if `"vocabulary.csv"` already exists to avoid redundant recomputation. If it does exist, the file is loaded; otherwise, a new vocabulary is created and saved.
    - **Unique Term Extraction**: Extracts all unique terms from the processed texts by flattening the list of lists and converting it to a set, ensuring only unique words are included.
    - **DataFrame Creation**: Each term is assigned a unique integer ID, and both the term and ID are stored in a DataFrame.
    - **CSV Storage**: Saves the vocabulary as `"vocabulary.csv"` to enable reuse in future computations.

In [43]:
# Import the get_vocabulary function from the search_engine module
from search_engine import get_vocabulary

# Generate or load the vocabulary DataFrame based on the 'processedDescription' column in df
descriptions_vocabulary_df = get_vocabulary(df['processedDescription'])

# Print the first few rows of the vocabulary DataFrame
print(descriptions_vocabulary_df.head().to_string(index=False))

Creating new vocabulary.csv file.
 term_id       term
       0        fun
       1     minori
       2    momento
       3        tre
       4 accomplish


Let us generate the **Inverted Index** dictionary, which maps each `term_id` to a list of document IDs where that term appears. Similar to the vocabulary, the function is designed to either create the `inverted_index.json` file if the dictionary has not been created yet or load it if it already exists, thereby avoiding unnecessary recomputation.

#### **Function**: `get_inverted_index`

- **Description:**  
    This function either creates a new **Inverted Index** or loads an existing one from a `.json` file. The inverted index maps `term_id` values to lists of document IDs where each corresponding term appears. If the index does not exist, it is created by iterating over the processed texts and populating the index with term-document mappings. The function avoids recomputation by saving the index to a `.json` file for future use.

- **Input**
    - `processed_texts` (list of list of str): A list of processed document texts, where each text is a list of terms (strings).
    - `vocabulary_df` (pandas.DataFrame): A DataFrame containing `term` and `term_id` columns, mapping each term to a unique `term_id`.
    - `file_path` (str, default=`"inverted_index.json"`): Path to the `.json` file where the inverted index will be stored or loaded from.

- **Output**
    - `inverted_index` (dict): A dictionary where keys are term IDs and values are lists of document indices (IDs) that contain each term.

- **Key Features**
    - **File Existence Check**: The function checks if an inverted index already exists in a `.json` file. If it does, the file is loaded; otherwise, a new index is created.
    - **Term Mapping**: A mapping of terms to their corresponding `term_id` is generated for fast lookups when building the index.
    - **Efficient Indexing**: The inverted index is constructed by iterating through the processed texts and storing document IDs for each unique term in a document.
    - **JSON Storage**: The inverted index is saved in a `.json` file to avoid recomputation in future runs, ensuring computational efficiency.


In [44]:
# Import the get_inverted_index function from the search_engine module
from search_engine import get_inverted_index

# Generate the inverted index using the processed descriptions and the descriptions vocabulary DataFrame
inverted_index = get_inverted_index(df['processedDescription'], descriptions_vocabulary_df)

# Iterate through the first 10 terms in the inverted index and print their corresponding document IDs
for idx, (term, docs) in enumerate(inverted_index.items()):
    if idx < 10:
        print(f"Term ID: {term}, Document IDs: {docs}")
    else:
        break

Creating Inverted Index...
Inverted Index saved to inverted_index.json.
Term ID: 0, Document IDs: [735, 1192, 1583]
Term ID: 1, Document IDs: [725]
Term ID: 2, Document IDs: [1450]
Term ID: 3, Document IDs: [733, 1143, 1248, 1763, 1767, 1883]
Term ID: 4, Document IDs: [894]
Term ID: 5, Document IDs: [443, 801]
Term ID: 6, Document IDs: [1468]
Term ID: 7, Document IDs: [886, 1849]
Term ID: 8, Document IDs: [874]
Term ID: 9, Document IDs: [1085, 1184]


#### 2.1.2 Execute the Query

Now, let us execute the query that will be handled by the **Conjunctive Search Engine**. This engine will process the query terms and return a list of restaurants whose descriptions contain all of the query words.

#### **Function**: `execute_conjunctive_query`
- **Description:**  
    This function processes a search query to find documents that contain all of the terms specified in the query. It uses the inverted index to identify the documents where each term appears and then returns only those documents that contain every term in the query.

- **Input:**
    - `query` (str): The search query, typically a string containing multiple terms.
    - `inverted_index` (dict): The inverted index, which maps `term_ids` to lists of document indices (IDs) where each term appears.
    - `vocabulary_df` (pd.DataFrame): A DataFrame mapping terms to their unique `term_ids`, allowing the function to look up terms' corresponding IDs.

- **Output:**
    - `intersection_result` (list of int): A list of document IDs that contain all of the terms specified in the search query.
    - `not_found` (str): A message listing the query terms that were not found in the vocabulary.

- **Key Features:**
    - **Preprocessing the Query:**  The input query is first preprocessed to tokenize and clean the terms before searching.
    - **Handling Missing Terms**: Any query terms that are not present in the vocabulary are filtered out, and a message is created to indicate which terms could not be matched.
    - **Mapping Terms to IDs:**  The query's remaining terms from the query are matched with their corresponding `term_ids` in the vocabulary DataFrame.
    - **Retrieving Document IDs:**  For each term in the query, the function fetches the document IDs where the term appears using the inverted index.
    - **Intersection of Document Sets:**  The function performs an intersection of all the document sets to ensure that only documents containing all the terms in the query are returned.

Let's execute the query `"modern seasonal cuisine"` and retrieve all restaurants whose processed descriptions contain all these words.

In [45]:
import pandas as pd
# Import the function for executing a conjunctive query based on the given terms
from search_engine import execute_conjunctive_query

# Execute the conjunctive query to get the document IDs that match all terms in the query
# The function returns a list of document IDs that contain all terms in the query ("modern seasonal cuisine")
documents_id, terms_not_found = execute_conjunctive_query(
    "modern seasonal cuisine",  # The query string containing terms to be matched
    inverted_index,  # The inverted index (term -> list of document IDs where each term appears)
    descriptions_vocabulary_df  # The vocabulary DataFrame that maps terms to their IDs
)

conjunctive_query_result_df = pd.DataFrame()

if len(documents_id) > 0:
    # Retrieve the documents that match the query by using the sorted document IDs
    # This creates a DataFrame with the restaurant details (name, address, description, website) for the matching documents
    conjunctive_query_result_df = df[['restaurantName', 'address', 'description', 'website']].iloc[sorted(documents_id)]

    # Rename the columns to make them more user-friendly for display purposes
    conjunctive_query_result_df = conjunctive_query_result_df.rename(columns={
        'restaurantName': 'Restaurant Name',
        'address': 'Address',
        'description': 'Description',
        'website': 'Website'
    })

# Print any terms from the query that were not found in the vocabulary
print(terms_not_found)
# Print the resulting DataFrame containing the restaurant details that match the query
conjunctive_query_result_df

,Restaurant Name,Address,Description,Website
104,Osteria Taviani,piazza Vittorio Emanuele II 28,"This pleasant, warmly decorated restaurant is ...",NaN
132,Cappuccini Cucina San Francesco,via Cappuccini 54,"Housed in the resort of the same name, this el...",https://www.cappuccini.it/
156,Pipero Roma,corso Vittorio Emanuele II 250,Situated opposite the church of Santa Maria in...,https://www.piperoroma.it/
193,Degusteria del Gigante,via degli Anelli 19,Situated in the charming historic centre of th...,https://www.degusteriadelgigante.it/
320,Babette,via Michelangelo 17,Situated just beyond the centre of Albenga in ...,https://www.ristorantebabette.net/
321,[àbitat],via Henry Dunant 1,"A young, enthusiastic and professional couple ...",https://www.abitatproject.it
345,20Tre,via David Chiossone 20 r,"Run by three partners, this contemporary-style...",https://www.ristorante20tregenova.it/
360,Secondo Tempo,via Vittorio Amedeo 55,Situated on the first floor of a building (for...,http://www.ristorantesecondotempo.it
405,Retrobottega,via della Stelletta 4,Minimalist decor and clean lines characterise ...,https://www.retro-bottega.com
509,Chichibio,via Guglielmo Marconi 1,"Despite its lack of awards, this restaurant st...",NaN


We illustrate an example where some terms in the query are not present in the vocabulary. When such a situation arises, the function processes the query using only the terms that match the vocabulary. Simultaneously, it notifies the user about the terms that could not be found. This method ensures that relevant results can still be retrieved, even when certain query terms are missing.

For this example, the query is `"modern cuisine seasonal albero"`. Notably, the results will be identical to the previous example, as only the terms `"modern"`, `"cuisine"`, and `"seasonal"` are found in the vocabulary and used for ranking. The term `"albero"` is absent from the vocabulary and therefore does not contribute to the results.

In [46]:
# Execute the conjunctive query to get the document IDs that match all terms in the query
# The function returns a list of document IDs that contain all terms in the query ("modern seasonal cuisine albero")
documents2_id, terms2_not_found = execute_conjunctive_query(
    "modern seasonal cuisine albero",  # The query string containing terms to be matched
    inverted_index,  # The inverted index (term -> list of document IDs where each term appears)
    descriptions_vocabulary_df  # The vocabulary DataFrame that maps terms to their IDs
)

conjunctive_query2_result_df = pd.DataFrame()

if len(documents2_id) > 0:
    # Retrieve the documents that match the query by using the sorted document IDs
    # This creates a DataFrame with the restaurant details (name, address, description, website) for the matching documents
    conjunctive_query2_result_df = df[['restaurantName', 'address', 'description', 'website']].iloc[sorted(documents_id)]

    # Rename the columns to make them more user-friendly for display purposes
    conjunctive_query2_result_df = conjunctive_query_result_df.rename(columns={
        'restaurantName': 'Restaurant Name',
        'address': 'Address',
        'description': 'Description',
        'website': 'Website'
    })

# Print any terms from the query that were not found in the vocabulary
print(terms2_not_found)
# Print the resulting DataFrame containing the restaurant details that match the query
conjunctive_query2_result_df

No matches found for these terms: albero


,Restaurant Name,Address,Description,Website
104,Osteria Taviani,piazza Vittorio Emanuele II 28,"This pleasant, warmly decorated restaurant is ...",NaN
132,Cappuccini Cucina San Francesco,via Cappuccini 54,"Housed in the resort of the same name, this el...",https://www.cappuccini.it/
156,Pipero Roma,corso Vittorio Emanuele II 250,Situated opposite the church of Santa Maria in...,https://www.piperoroma.it/
193,Degusteria del Gigante,via degli Anelli 19,Situated in the charming historic centre of th...,https://www.degusteriadelgigante.it/
320,Babette,via Michelangelo 17,Situated just beyond the centre of Albenga in ...,https://www.ristorantebabette.net/
321,[àbitat],via Henry Dunant 1,"A young, enthusiastic and professional couple ...",https://www.abitatproject.it
345,20Tre,via David Chiossone 20 r,"Run by three partners, this contemporary-style...",https://www.ristorante20tregenova.it/
360,Secondo Tempo,via Vittorio Amedeo 55,Situated on the first floor of a building (for...,http://www.ristorantesecondotempo.it
405,Retrobottega,via della Stelletta 4,Minimalist decor and clean lines characterise ...,https://www.retro-bottega.com
509,Chichibio,via Guglielmo Marconi 1,"Despite its lack of awards, this restaurant st...",NaN


### 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity

Finally, we build the **Ranked Search Engine**, that returns the *top-k* restaurants sorted by their similarity to the query, utilizing **TF-IDF** and **Cosine Similarity**.

#### 2.2.1 Inverted Index with TF-IDF Scores

Let us obtain the **Inverted Index with TF-IDF Scores** dictionary. 

Like in previous cases, the following function creates the file `tfIdf_inverted_index.json` if it doesn't exist. Otherwise, it loads the file, avoiding recomputation, and stores the data in a dictionary. This is a new inverted index where each entry corresponds to a term, and the value is a list of tuples containing document IDs and their associated TF-IDF scores.

#### **Function**: `get_tfIdf_inverted_index` 
- **Description**:\
    This function creates or loads an inverted index with **TF-IDF scores** for a collection of documents. It calculates the importance of each term in each document based on the **term frequency (TF)** and **inverse document frequency (IDF)**. If the TF-IDF inverted index already exists as a JSON file, it will be loaded; otherwise, the function will generate it and save it to a file for future use.

- **Input**:
    - `inverted_index` (dict): A dictionary where the keys are `term_ids`, and the values are lists of document IDs (indices) where each term appears.
    - `vocabulary_df` (pd.DataFrame): A DataFrame that maps terms to their unique `term_ids`. This is used to look up terms and their IDs.
    - `processed_texts` (list of list of str): A list of processed documents, where each document is represented as a list of terms (strings).
    - `file_path` (str, default=`"tfIdf_inverted_index.json"`): The file path where the TF-IDF inverted index will be saved or loaded. Default is `"tfIdf_inverted_index.json"`.

-  **Output**:
    - **tfIdf_inverted_index** (dict): A dictionary where the keys are `term_ids`, and the values are lists of tuples. Each tuple contains:
        - `doc_id` (int): The document ID (index in the `processed_texts` list).
        - `tf-idf score` (float): The TF-IDF score for the term in the corresponding document.

- **Key Features**:
    - **Check for Existing File**: The function checks whether the TF-IDF inverted index already exists as a JSON file. If it does, it loads the index from the file to avoid recomputation.
    - **Calculate TF-IDF Scores**: If the file does not exist, the function computes the TF-IDF scores for each term in the vocabulary, using the `inverted_index` and `processed_texts` as input.
    - **Save the Index**: Once the TF-IDF scores are calculated, the index is saved to a JSON file for future use, ensuring efficient reuse of precomputed data.


### **Function**: `get_tfIdf` 
- **Description**:\
    This function calculates the **TF-IDF (Term Frequency-Inverse Document Frequency)** score for a specific term in a document, providing a measure of the term's importance within that document relative to the entire corpus. It combines two components:
    
    - **Term Frequency (TF)**: The frequency of the term within the document.
    - **Inverse Document Frequency (IDF)**: A measure of the rarity of the term across the entire corpus.

    The function is used in `get_tfIdf_inverted_index` to compute the TF-IDF scores for terms in documents and build the **TF-IDF inverted index**.

- **Input**:
    - `term` (str): The term for which the TF-IDF score is being calculated.
    - `document` (list of str): A list of terms representing the document being analyzed.
    - `corpus` (list of list of str): A collection of documents, where each document is represented as a list of terms.

- **Output**:
    - **TF-IDF score** (float): The TF-IDF score for the given term in the specified document. It is the product of the **TF** and **IDF** scores.

- **Key Features**:
    - **Term Frequency (TF)**: The function calculates the term frequency as the ratio of occurrences of the term in the document to the total number of terms in the document.
    - **Inverse Document Frequency (IDF)**: The function computes the IDF by taking the logarithm of the total number of documents divided by the number of documents that contain the term, ensuring that terms appearing in every document are not given excessive weight.
    - **Logarithmic Scaling**: The IDF is logarithmic to reduce the effect of very frequent terms in the corpus.
    - **Combination of TF and IDF**: The TF and IDF values are multiplied together to produce the TF-IDF score, which reflects the relative importance of the term in the document within the context of the entire corpus.

In [47]:
# Importing the get_tfIdf_inverted_index function from the search_engine module
from search_engine import get_tfIdf_inverted_index

# Create the TF-IDF inverted index dictionary using the inverted index, vocabulary, and processed descriptions
tfIdf_inverted_index = get_tfIdf_inverted_index(inverted_index, descriptions_vocabulary_df, df['processedDescription'])

# Print the first 10 key-value pairs from the TF-IDF inverted index dictionary
for idx, (term, docs) in enumerate(tfIdf_inverted_index.items()):
    if idx < 10:
        print(f"Term ID: {term}, List of (Doc ID, TF-IDF score) pairs: {docs}")
    else:
        break

Creating Inverted Index with TF-IDF scores...
Inverted index with TF-IDF scores saved to tfIdf_inverted_index.json.
Term ID: 0, List of (Doc ID, TF-IDF score) pairs: [(735, np.float64(0.036416547083926824)), (1192, np.float64(0.04646249110707904)), (1583, np.float64(0.11716628192219933))]
Term ID: 1, List of (Doc ID, TF-IDF score) pairs: [(725, np.float64(0.03941913789308639))]
Term ID: 2, List of (Doc ID, TF-IDF score) pairs: [(1450, np.float64(0.06512727130162099))]
Term ID: 3, List of (Doc ID, TF-IDF score) pairs: [(733, np.float64(0.022288967595675363)), (1143, np.float64(0.034052589382281805)), (1248, np.float64(0.032690485806990535)), (1763, np.float64(0.03891724500832206)), (1767, np.float64(0.06286631885959719)), (1883, np.float64(0.03502552050748986))]
Term ID: 4, List of (Doc ID, TF-IDF score) pairs: [(894, np.float64(0.05447008145226483))]
Term ID: 5, List of (Doc ID, TF-IDF score) pairs: [(443, np.float64(0.03278794442812656)), (801, np.float64(0.023498026840157372))]
Term 

#### 2.2.2 Execute the Ranked Query

In this step, we will process the query entered by the user and utilize a **Ranked Search Engine** to return the most relevant documents (restaurants) based on their similarity to the query. The search engine will return the ***top-k*** matching restaurants with the highest similarity scores. If fewer than *k* restaurants have a non-zero similarity score, it will return all the restaurants that have any similarity to the query.

### **Function**: `execute_ranked_query`

- **Description:**\
    This function performs a ranked search by evaluating how closely the documents in a collection match the query based on the **TF-IDF** vectors. It uses only the terms from the query that are present in the **vocabulary**, and the **cosine similarity** is computed to rank the documents.

- **Input**:
    - `query_terms` (str): A space-separated string of terms that represent the search query.
    - `tfIdf_inverted_index` (dict): A dictionary mapping term IDs to lists of tuples (document ID, TF-IDF score), representing the inverted index for the documents.
    - `vocabulary_df` (DataFrame): A DataFrame containing terms and their corresponding term IDs.
    - `processed_texts` (list of list of str): A list of processed documents, where each document is represented as a list of terms.
    - `top_k` (int): The number of top-ranked documents to return based on similarity.

- **Output**:
    - `ranked_results` (list): A list of tuples, where each tuple contains a document ID and its cosine similarity score with respect to the query.
    - `not_found` (str): A message listing the query terms that were not found in the vocabulary.

- **Key Features:**
    - **Preprocessing the Query**: The input query, a string of space-separated terms, is tokenized and cleaned using the `preprocess_text` function to ensure that the query terms are in a standard form for comparison.
    - **Handling Missing Terms**: Any query terms that are not present in the vocabulary are filtered out, and a message is created to indicate which terms could not be matched.

    - **Mapping Query Terms to Term IDs**: The remaining valid query terms are mapped to their corresponding **term IDs** using the **vocabulary** DataFrame. These term IDs will be used to build the **query vector**.
    
    - **Building the Query Vector**: A **query vector** is initialized with zeros, and for each term in the query, the **TF-IDF** score is calculated using the `get_tfIdf` function. These scores populate the query vector at the positions corresponding to the term IDs.
    
    - **Building Document Vectors**: Document vectors are also initialized with zero values. These vectors are populated using the **TF-IDF scores** from the **TF-IDF inverted index**, which stores the scores for each term in each document.

    - **Cosine Similarity Calculation**: For each document, the cosine similarity between its vector and the query vector is computed using the `get_cosine_similarity` function. Cosine similarity measures the angle between the two vectors, providing a score that indicates how similar the document is to the query.

    - ***top-k* Ranking**: The documents are ranked based on their similarity scores, with the most similar documents appearing first. The results are sorted in descending order of similarity. If there are more than `top_k` results, the function limits the returned documents to the top-k highest ranked documents. If fewer than `top_k` documents match, all matching documents are returned.

### **Function**: `get_cosine_similarity`   
- **Description:**
    This function is a helper function used by the `execute_ranked_query` function to calculate the **cosine similarity** between a document vector and a query vector. Cosine similarity is calculated as the cosine of the angle between two vectors. A cosine similarity score of:
    - **1** means the vectors are identical.
    - **0** means the vectors are orthogonal (no similarity).
    - **-1** indicates complete dissimilarity (opposites).
    
    This function specifically handles cases where one or both vectors might be zero vectors, returning 0 if the cosine similarity cannot be calculated.

- **Input**:
    - `doc_vector` (numpy array): A vector representing the document, containing **TF-IDF scores** for each term.
    - `query_vector` (numpy array): A vector representing the query, with **TF-IDF scores** assigned for each term.

- **Output**:
    - `float`: The cosine similarity score between the document and query vectors. Returns 0 if the denominator (product of norms) is zero.

- **Key Features**:
    - **Dot Product Calculation**: The **dot product** of the document and query vectors is computed. This represents the overlap or alignment between the two vectors.

    - **L2 Norm Calculation**: The **L2 norm** (magnitude) of both the document vector and the query vector is calculated individually. The L2 norm provides a measure of the vector's length.

    - **Cosine Similarity Calculation**: The cosine similarity score is derived by dividing the dot product by the product of the norms of the two vectors. This step effectively normalizes the dot product, yielding a similarity score between -1 and 1.

    - **Handling Zero Vectors**: If either vector is a **zero vector** (i.e., the L2 norm is zero), the function returns 0, as the cosine similarity is undefined in this case.

Let's execute the query `"modern seasonal cuisine"` and retrieve the *top-10* results ranked by relevance. This will allow us to see the best-matching documents based on cosine similarity between the query vector and document vectors. Any terms from the query that are not found in the vocabulary will be noted in the output.

In [48]:
# Import the function for executing a ranked query based on TF-IDF scores
from search_engine import execute_ranked_query

# Execute the ranked query with the search terms "modern seasonal cuisine"
# The function returns the ranked query result (documents and their similarity scores) and any terms not found in the vocabulary
ranked_query_result, terms_not_found = execute_ranked_query(
    "modern seasonal cuisine",  # The query string
    tfIdf_inverted_index,  # The TF-IDF inverted index (term -> document ID, TF-IDF score pairs)
    descriptions_vocabulary_df,  # The vocabulary DataFrame (terms and their IDs)
    df['processedDescription'],  # The processed descriptions of the documents
    10  # Number of top results to return
)

ranked_query_result_df = pd.DataFrame()

if len(ranked_query_result) > 0:

    # Extract the document IDs from the ranked query result
    ranked_query_result_ids = [doc_id for (doc_id, _) in ranked_query_result]

    # Extract the TF-IDF scores from the ranked query result
    ranked_query_result_scores = [tfIdf_score for (_, tfIdf_score) in ranked_query_result]

    # Retrieve the restaurant information (name, address, description, and website) based on the document IDs
    ranked_query_result_df = df[['restaurantName', 'address', 'description', 'website']].iloc[ranked_query_result_ids]

    # Add the TF-IDF similarity scores as a new column in the DataFrame
    ranked_query_result_df['tfIdf_score'] = ranked_query_result_scores

    # Rename the columns to make them more user-friendly for display purposes
    ranked_query_result_df = ranked_query_result_df.rename(columns={
        'restaurantName': 'Restaurant Name',
        'address': 'Address',
        'description': 'Description',
        'website': 'Website',
        'tfIdf_score': 'Similarity Score'
    })

# Print any terms from the query that were not found in the vocabulary
print(terms_not_found)

# Display the DataFrame with the top-ranked results, including the restaurant details and similarity scores
ranked_query_result_df


,Restaurant Name,Address,Description,Website,Similarity Score
1340,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, al...",https://ristorantesaur.it,0.250021
105,La Botte,via Giuseppe Garibaldi 8,A modern and welcoming contemporary bistro sit...,http://www.trattorialabottestresa.it,0.236478
1658,Razzo,via Andrea Doria 17/f,"A quiet restaurant with a relaxed, young and m...",https://vadoarazzo.it/,0.226028
655,Piccolo Lord,corso San Maurizio 69 bis/g,"Professional service in a welcoming, modern re...",https://www.ristorantepiccololord.it/,0.217682
1752,La Valle,"via Umberto I 25, località Valle Sauglio",A well - run restaurant in a quiet area just o...,https://www.ristorantelavalle.it/,0.205406
979,Al Vecchio Convento,viale Borri 348,"Ask for a table in the main dining room, with ...",https://www.alvecchioconvento.it/,0.189035
1134,RistoFante,via Mazzini 41,The motto of this restaurant is “In step with ...,https://www.ristofante.it/,0.183209
1593,Aprudia,largo del Forno 16,"At this restaurant in the historic centre, whe...",http://www.aprudia.com,0.173982
476,Barbieri,via Italo Barbieri,Enjoy your meal in the classic - style dining ...,https://www.hotelbarbieri.it,0.173446
1886,Guallina,"via Molino Faenza 19, località Guallina",Situated in a small house in an outlying villa...,http://www.trattoriaguallina.it,0.168042


Here, we demonstrate an example where some terms in the query are not present in the vocabulary. When this occurs, the function will return ranked results based solely on the terms that do match the vocabulary, while also notifying the user of the terms that were not found. This approach allows us to retrieve relevant results even if certain query terms are missing from the vocabulary.

In this example, the query is `"modern albero cuisine seasonal treno"`. Note that the results will be identical to the previous example, as only the terms `"modern"`, `"cuisine"`, and `"seasonal"` were found and used for ranking, while `"albero"` and `"treno"` were absent from the vocabulary.

In [49]:
ranked_query2_result, terms2_not_found = execute_ranked_query(
    "modern albero cuisine seasonal treno", # The query string
    tfIdf_inverted_index, # The TF-IDF inverted index (term -> document ID, TF-IDF score pairs)
    descriptions_vocabulary_df, # The vocabulary DataFrame (terms and their IDs)
    df['processedDescription'], # The processed descriptions of the documents
    10 # Number of top results to return
)

ranked_query2_result_df = pd.DataFrame()

if len(ranked_query2_result) > 0:
    ranked_query2_result_ids = [doc_id for (doc_id, _) in ranked_query2_result]

    # Extract the TF-IDF scores from the ranked query result
    ranked_query2_result_scores = [tfIdf_score for (_, tfIdf_score) in ranked_query2_result]

    # Retrieve the restaurant information (name, address, description, and website) based on the document IDs
    ranked_query2_result_df = df[['restaurantName', 'address', 'description', 'website']].iloc[ranked_query2_result_ids]

    # Add the TF-IDF similarity scores as a new column in the DataFrame
    ranked_query2_result_df['tfIdf_score'] = ranked_query2_result_scores

    # Rename the columns to make them more user-friendly for display purposes
    ranked_query2_result_df = ranked_query2_result_df.rename(columns={
        'restaurantName': 'Restaurant Name',
        'address': 'Address',
        'description': 'Description',
        'website': 'Website',
        'tfIdf_score': 'Similarity Score'
    })

print(terms2_not_found)
# Display the DataFrame with the top-ranked results, including the restaurant details and similarity scores
ranked_query2_result_df


No matches found for these terms: treno, albero


,Restaurant Name,Address,Description,Website,Similarity Score
1340,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, al...",https://ristorantesaur.it,0.250021
105,La Botte,via Giuseppe Garibaldi 8,A modern and welcoming contemporary bistro sit...,http://www.trattorialabottestresa.it,0.236478
1658,Razzo,via Andrea Doria 17/f,"A quiet restaurant with a relaxed, young and m...",https://vadoarazzo.it/,0.226028
655,Piccolo Lord,corso San Maurizio 69 bis/g,"Professional service in a welcoming, modern re...",https://www.ristorantepiccololord.it/,0.217682
1752,La Valle,"via Umberto I 25, località Valle Sauglio",A well - run restaurant in a quiet area just o...,https://www.ristorantelavalle.it/,0.205406
979,Al Vecchio Convento,viale Borri 348,"Ask for a table in the main dining room, with ...",https://www.alvecchioconvento.it/,0.189035
1134,RistoFante,via Mazzini 41,The motto of this restaurant is “In step with ...,https://www.ristofante.it/,0.183209
1593,Aprudia,largo del Forno 16,"At this restaurant in the historic centre, whe...",http://www.aprudia.com,0.173982
476,Barbieri,via Italo Barbieri,Enjoy your meal in the classic - style dining ...,https://www.hotelbarbieri.it,0.173446
1886,Guallina,"via Molino Faenza 19, località Guallina",Situated in a small house in an outlying villa...,http://www.trattoriaguallina.it,0.168042


## 3. **Define a New Score!**

We need to extract:

-Description Terms: Words relevant to the description (e.g., "Italian").

-Cuisine Type: "Italian"

-Facilities and Services: ["Outdoor seating", "Wi-Fi"]

-Price Range: "Moderately priced"(€)

In [54]:
import new_score
import importlib
importlib.reload(new_score)

import numpy as np


 we are going to calculate a custom score for a restaurant based on multiple attributes and preferences. The attributes are:
 
 - Description match 

- Cuisine match

- Facilities/services match

- Price range match



In [ ]:
from new_score import compute_custom_new_score
from new_score import get_top_k_restaurants
from new_score import execute_ranked_query1

The `function compute_custom_new_score` assigns weights to each attribute and combines them into a final score, in particular it uses this parameters:

- doc_id: The ID of the restaurant to be scored.

- description_scores: dictionary containing document ID and their similarity scores for the match.

- df (Dataframe): A pandas DataFrame containing details about all restaurants.

- user_cuisines: The list of cuisines, input by the user.

- user_facilities: The list of desired facilities/services, input by the user.

- user_price: The user preferred price range ('€', '€€', '€€€').

- max_description_score: The highest similarity score for normalizing 
      description match scores.

it returns:

- float: The total custom score for the restaurant.

The function `get_top_k_restaurants` processe a ranked list of restaurants  and uses a the new scoring function, mentioned before, (compute_custom_new_score) to evaluate each restaurant based on the input preferences.

Explanation:

-The `description_scores` in `ranked_docs` are normalized by dividing each score by the maximum score (`max_description_score`). This ensures all scores are between 0 and 1.

-Calls `compute_custom_new_score` to calculate a new score for the restaurant, using the description similarity, cuisine, facilities and price match based on user's input preferences.

-Manage the Min-Heap:
If the heap contains fewer than k elements, the current restaurant is added.
If the heap already contains k elements, the low scoring restaurant is replaced if the current restaurant with an higher score.

$\rightarrow$ by using a heap, the function avoids sorting all restaurants, giving an higher performance when working with large datasets like this case.

-once all restaurants are processed, the heap is converted into a sorted list (in descending order) and returned.


In [56]:
#user inputs for searchinng the restaurants 
user_description_query = 'seasonal modern cuisine'
user_cuisine_query = 'Italian'
user_facilities_query = 'wheelchair access, air conditioning'
user_price_query = '€€'


In [57]:

#parse user inputs
user_cuisines = [c.strip().lower() for c in user_cuisine_query.split(',')]
user_facilities = [f.strip().lower() for f in user_facilities_query.split(',')]
user_price = user_price_query.strip()

# initial ranked query based on description
ranked_query_result, terms_not_found = execute_ranked_query1(
    user_description_query,
    tfIdf_inverted_index,
    descriptions_vocabulary_df,
    df['processedDescription'],
    top_k=None  # Get all documents
)

#get top-k restaurants using the new custom score
top_k = 10
top_restaurants = get_top_k_restaurants(
    ranked_query_result,
    df,
    user_cuisines,
    user_facilities,
    user_price,
    k=top_k
)

# display restaurant details
restaurant_ids = [doc_id for (_, doc_id) in top_restaurants]
scores = [score for (score, _) in top_restaurants]

# build DataFrame
result_df = df.loc[restaurant_ids, ['restaurantName', 'address', 'description', 'website']]
result_df['Custom Score'] = scores

#sort by score descending
result_df = result_df.rename(columns={
    'restaurantName': 'Restaurant Name',
    'address': 'Address',
    'description': 'Description',
    'website': 'Website'
})

#display the sorted results


result_df.sort_values(by='Custom Score', ascending=False)


,Restaurant Name,Address,Description,Website,Custom Score
870,L'Ortone,piazza Lorenzo Ghiberti 87 r,This Tuscan-style bistro next to Sant'Ambrogio...,https://www.lortone.it/,0.780007
208,Da Giacomo,piazza Municipio 2,This is a small restaurant in the historic cen...,https://www.dagiacomo.it,0.767160
1340,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, al...",https://ristorantesaur.it,0.750000
998,Buriani dal 1967,via Matteotti 66,A family-run restaurant in the province of Bol...,https://www.ristoranteburiani.com/,0.735722
105,La Botte,via Giuseppe Garibaldi 8,A modern and welcoming contemporary bistro sit...,http://www.trattorialabottestresa.it,0.731042
1592,Duo,via Senatore Dallorso 10,In this restaurant housed in a historic palazz...,NaN,0.725835
1658,Razzo,via Andrea Doria 17/f,"A quiet restaurant with a relaxed, young and m...",https://vadoarazzo.it/,0.716412
215,Il Galeone,piazzale Amendola 2,Housed on the ground floor of the Elisabeth Du...,https://www.ilgaleone.net/,0.709198
1648,Nascostoposto,via Sant'Alò,Easily found despite its name (nascosto posto ...,NaN,0.705218
107,85 Bistrot,piazza Martiri di Via Fani 85,"This simple, modern, family-run trattoria with...",https://www.85bistrotsesto.com/,0.700470


so here the principal steps of the code:

-take the user inputs

-use the preview TD-IDF search to get a list of revelant restaurants

-calculate the new score using the function 'compute_custom_new_score'

-heap manage the top k restaurants

-display the results

## 5. **Bonus**


In [58]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.6 MB/s eta 0:00:00a 0:00:01


In [60]:
!pip install tabulate

In [61]:
import bonus

#### **Function**: `compute_query_similarity`
- **Description**: 
    Computes the cosine similarity between a query vector (TF-IDF) and document vectors using only the terms that exist in the vocabulary. 
    This is done by processing the input query, filtering the terms, creating a query vector, and calculating the similarity scores for each document.
- **Input**:
    - `query_terms` (str): Query input as a space-separated string of terms.
    - `inverted_index` (dict): Dictionary where each key is a term ID, and each value is a list of tuples (document ID, TF-IDF score) representing the inverted index for documents.
    - `vocabulary_df` (DataFrame): DataFrame containing the vocabulary terms and their corresponding term IDs.
    - `processed_texts` (list of list of str): List of preprocessed texts, where each text is a list of terms.
- **Output**:
    - Returns a NumPy array of cosine similarity scores for each document.  
- **Uses the following functions from `search_engine.py`**:
- `preprocess_text`: To clean and preprocess the query terms.
- `get_tfIdf`: To retrieve the TF-IDF values for the terms in the query.
- `cosine_similarity`: To compute the cosine similarity between the document vectors and the query vector.


   #### **Function**: `prepare_search_data`
   - **Description**: 
     Prepares the necessary data for search functionality by processing the restaurant name, city, and cuisine type, creating vocabularies, inverted indices, and TF-IDF representations for each of these attributes.
   - **Input**:
     - `df`: A DataFrame containing the restaurant data with columns for `restaurantName`, `city`, and `cuisineType`.
   - **Output**:
     - Returns a tuple containing:
       - `tfidf_list`: A list of TF-IDF inverted index for the restaurant name, city, and cuisine.
       - `voc_list`: A list of vocabulary mappings for the name, city, and cuisine.
       - `processed_list`: A list of processed text data for restaurant name, city, and cuisine.
  - **Uses the following functions from `search_engine.py`:**
      - `preprocess_text`: To clean and preprocess text data.
      - `create_vocabulary`: To create vocabulary for each text column.
      - `create_inverted_index`: To create inverted indices for efficient search.
      - `create_tfIdf_inverted_index`: To calculate TF-IDF values for the processed columns.


In [62]:
tfidf_list, voc_list, processed_list = bonus.prepare_search_data(df)

Loading existing vocabulary file for the field: name.
Loading existing vocabulary file for the field: city.
Loading existing vocabulary file for the field: cuis.
Loading inverted index from file for the field: name.
Loading inverted index from file for the field: city.
Loading inverted index from file for the field: cuis.
Loading inverted index with TF-IDF scores from file for the field: name.
Loading inverted index with TF-IDF scores from file for the field: city.
Loading inverted index with TF-IDF scores from file for the field: cuis.


#### **Function**: `multiple_ranked_query`
- **Description**: 
    Executes ranked queries for multiple fields (name, city, and cuisine) with weighted importance. It calculates cosine similarity for each field-specific query and filters the documents based on the weighted scores across all fields.
- **Input**:
    - `query_list` (list of str): A list of field-specific queries, where each query corresponds to a specific field (e.g., name, city, cuisine).
    - `tfidf_list` (list of dict): A list of TF-IDF inverted indices, each corresponding to one field.
    - `voc_list` (list of DataFrame): A list of vocabulary DataFrames for the terms of each field.
    - `processed_list` (list of list of str): A list of preprocessed terms for each field's query.
    - `df` (DataFrame): The dataset of restaurants, containing details such as name, city, and cuisine type.
- **Output**:
    - Returns a filtered DataFrame containing only restaurants with non-zero scores. Includes a new column, `score`, with the weighted similarity score across all fields.
- **Key Features**:
    - Uses `compute_query_similarity` to calculate the cosine similarity between the query vector and document vectors for each field.  
    - Computes cosine similarity for each query field and applies field-specific weights, prioritizing name, then city, and lastly cuisine type matches. 
    - Adds a new `score` column to indicate the relevance of each restaurant.
    - Based on the function `execute_ranked_query` from `search_engine.py`, which provides the foundation for ranking the results by relevance.



#### **Function**: `display_results_and_filters`
- **Description**: 
    Displays restaurant search results with interactive filters, allowing users to refine the results based on region, price range, facilities, and credit card options. The function dynamically updates the displayed results as filter selections are changed.
- **Input**:
    - `df_res` (DataFrame): DataFrame containing restaurant details such as name, address, city, region, price range, cuisine type, and facilities.
    - `top_k` (int, optional): The maximum number of top-ranked restaurants to display. Defaults to 10.
- **Output**:
    - None: Results and filter widgets are displayed interactively in a Jupyter Notebook environment.
- **Key Features**:
    - Displays an interactive UI with filters for price range, region, facilities, and credit cards.
    - Dynamically updates the displayed results based on filter selections.
    - Filters restaurants by selected price range, region (inclusive filter), facilities, and credit card options (exclusive filters).
    - Efficiently computes the top `top_k` restaurants using a heap structure for fast retrieval of the highest-ranked results.
    - Shows the top `top_k` restaurants, ranked by their relevance score, with key details such as name, address, and cuisine.


#### **Function**: `search`
- **Description**: 
    Sets up a search interface for users to find restaurants based on their input. Users can search by restaurant name, city, or cuisine type, and control the number of results displayed. Once the user enters their search criteria and selects the number of results, the function processes the query and displays the results using an interactive UI with filters.
  
- **Input**:
    - `tfidf_list` (list): List containing the TF-IDF vectors for the restaurant data.
    - `voc_list` (list): Vocabulary list associated with the TF-IDF model.
    - `processed_list` (list): List of pre-processed restaurant data.
    - `df` (DataFrame): The DataFrame containing the restaurant dataset.
  
- **Output**:
    - Displays an interactive search interface and filtered results, showing the top-ranked restaurants based on the search query and applied filters.

- **Key Features**:
    - **Interactive search**: Users can input values for restaurant name, city, and cuisine type using text input fields.
    - **Results control**: Users can choose how many results to display (10, 20, 50, 100) through a dropdown menu.
    - **Multiple field search**: The search can be based on any combination of restaurant name, city, or cuisine type.
    - **Query processing**: calls `multiple_ranked_query` to process the search query and retrieve relevant results with their scores. 
    - **Results display**: calls `display_results_and_filters` to filter and display the ranked results.


In [63]:
bonus.search(tfidf_list, voc_list, processed_list, df)

IntRangeSlider(value=(1, 4), continuous_update=False, description='Price Range:', max=4, min=1, style=SliderSt…

Label(value='Filter by Region')

GridBox(children=(Checkbox(value=False, description='Abruzzo'), Checkbox(value=False, description='Aosta Valle…

Label(value='Filter by Services')

GridBox(children=(Checkbox(value=False, description='Air conditioning'), Checkbox(value=False, description='Br…

Label(value='Filter by Credit Cards')

GridBox(children=(Checkbox(value=False, description='amex'), Checkbox(value=False, description='dinersclub'), …

Output()

## **Algorithmic Question**

### 1. Write the pseudocode for an algorithm that solves this problem.

Below, we provide the pseudocode for the requested algorithm. This pseudocode outlines the logic used to determine whether all packages in a grid can be collected and, if possible, generates the lexicographically smallest path.

**Function** *algorithm(t, n, grids)*

**Input:**  
- *t*: Number of test cases $(1\leq t \leq 10)$
- *n*: Number of packages $(1\leq n \leq 100)$.
- *grids*: A sequence of $t$ test cases. Each $j^{th}$ test case, for $j=1, \dots, t$, is represented as a sequence of coordinates (a *grid*), where each coordinate is a pair $(x_i, y_i)$ representing the position of the $i^{th}$ package, for $i=1, \dots, n$, and $0 \leq x_i, y_i \leq 100$.

**Output:**  The result for each test case, that can be:
  - *"YES"* followed by the lexicographically smallest path, where *R* denotes moving to the right and *U* denotes moving upward.
  - Or *"NO"* if it is impossible to collect all the packages.

    *results* <- $\emptyset$\
    *n_steps* <- $0$
    
    ***for*** *i* = 0 to *t*:  
    &emsp; *grid* ← *grids*[*i*]  
    &emsp; *grid* ← *grid* $\cup$ $\{(0,0)\}$  
    &emsp; ***Sort*** *grid* by *x*-coordinate first, then by *y*-coordinate  
    &emsp; *path* ← $\emptyset$  

    &emsp; ***for*** *j* = 0 to *n* - 1:  
    &emsp;&emsp; *result* ← $\emptyset$  
    &emsp;&emsp; Let *p* be the nearest next package  
    &emsp;&emsp; ***if*** *p* is down and/or to the left ***then***:  
    &emsp;&emsp;&emsp; *result* ← {"NO"} and ***break***  

    &emsp;&emsp; ***else***:  
    &emsp;&emsp;&emsp; ***if*** *p* is on the right ***then***:  
    &emsp;&emsp;&emsp;&emsp; *n\_steps* ← number of steps to the right to reach *p*  
    &emsp;&emsp;&emsp;&emsp; ***Add*** "R" *n\_steps* times to *path*  

    &emsp;&emsp;&emsp; ***if*** *p* is up ***then***:  
    &emsp;&emsp;&emsp;&emsp; *n\_steps* ← number of up steps to reach *p*  
    &emsp;&emsp;&emsp;&emsp; ***Add*** "U" *n\_steps* times to *path*  
    
    &emsp;&emsp;&emsp; *result* ← {"YES"} $\cup$ *path*  
    
    &emsp;&emsp; *results* <- *results* $\cup$ *result*

    ***return*** *result*

The algorithm has been implemented in the module `algorithmic_question` under the function `algorithm`. We will now test this implementation using the proposed input, which is provided in the file `input_algorithmic_question.txt`.

In [20]:
from algorithmic_question import algorithm

# Open the input file "input.txt" in read mode
with open("input_algorithmic_question.txt", 'r') as file:
    # Read the first line which indicates the number of test cases (t)
    t = int(file.readline())  # Number of test cases
    
    grid_list = []  # Initialize an empty list to store the grids for each test case
    
    # Loop through each test case (from 1 to t)
    for _ in range(t):
        # Read the number of packages (n) for the current test case
        n = int(file.readline())  # Number of packages in this test case
        grid = []  # Initialize an empty list to store the coordinates of packages in the current grid
        
        # Read the coordinates for each package in the test case
        for _ in range(n):
            # Read the x and y coordinates of the package, split by space, and convert them to integers
            x, y = map(int, file.readline().split())
            # Append the package coordinates as a list [x, y] to the grid
            grid.append([x, y])
        
        # Add the current grid (list of package coordinates) to the grid_list
        grid_list.append(grid)

    # After collecting all test cases, pass the grid_list to the algorithm function
    result = algorithm(t, n, grid_list)
    
    # Print the result returned by the algorithm function (either "YES" with the path or "NO")
    print(result)

YES
RUUURRRRUU
NO
YES
RRRRUUU



### 2. Prove that your algorithm is correct.

To prove the correctness of the algorithm, we need to show that it correctly identifies whether it is possible to visit all packages starting from $(0,0)$ using only moves to the right *R* and up *U*, and constructs the **lexicographically smallest path** if a valid path exists.

The robot can only move to the right *R* or up *U*, and to ensure a valid path, the algorithm guarantees that the robot never needs to move left or down. This is achieved by sorting the packages first by $x$ in ascending order and then by $y$ in ascending order to break ties. Sorting in this way ensures that the robot never moves left because $x_{i+1} \geq x_i$ is automatically satisfied. To check that the robot does not move down, the algorithm verifies that $y_{i+1} \geq y_i$ for every consecutive package. If this condition is violated, the algorithm concludes that a valid path is not possible and returns "NO"; otherwise, it constructs the path using only moves to the right *R* and up *U*.

Given the limited domain of moves available to the robot (only rightward and upward), there are only two logical approaches to solve the problem:
1. Sorting packages by $x$ first and then by $y$, ensuring the robot moves consistently right and up.
2. Sorting packages by $y$ first and then by $x$, which similarly ensures a valid path if no downward or leftward moves are required.

Among these, sorting by $x$ first and then by $y$ has the additional advantage of producing the **lexicographically smallest path**. By always completing all rightward moves before upward moves for any pair of points $(x_i, y_i)$ and $(x_{i+1}, y_{i+1})$, the resulting string of moves is alphabetically smallest. For instance, given the packages $(0,0), (2,3), (1,1)$, sorting by $x$ first produces the path `RURUU`, while sorting by $y$ first produces a different valid path `URURR`, which is not lexicographically smallest.

The total number of moves required to collect all packages is invariant under different sorting orders, as it is determined by the Manhattan distances between the packages:
\begin{equation*}
\text{Total moves} = \sum_{i=0}^{n-1} \left( (x_{i+1} - x_{i}) + (y_{i+1} - y_{i}) \right).
\end{equation*}
This means that the robot's path will always have the same total length regardless of the sorting method, provided the path is valid. However, sorting by $x$ first and then by $y$ ensures both validity and the lexicographically smallest path, as it prioritizes rightward moves *R* over upward moves *U*.

In conclusion, the algorithm guarantees a valid path if one exists by sorting packages in ascending order of $x$ and then $y$, ensuring no leftward or downward moves. It constructs the lexicographically smallest path by always prioritizing *R* over *U*, minimizing the total number of moves as required by the constraints. These two sorting approaches are the only possible ways to ensure validity given the domain of moves, with the $x$-first approach being optimal for producing the smallest path lexicographically.

### 3. Compute the time complexity of your algorithm in Big O notation. Break down the steps involved in the algorithm, and explain which parts contribute most to the overall time complexity.

This analysis provides a detailed breakdown of the algorithm's steps and their contributions to the overall time complexity. The focus is on understanding the operations that dominate the computational cost and their implications for efficiency.

#### Steps of the Algorithm per Test Case

1. **Including the Origin Point**: The algorithm starts by adding the origin $(0, 0)$ to the set of package coordinates. This is a simple operation that does not depend on the size of the input and takes constant time $O(1)$. Since the sequence’s order is irrelevant at this stage, no additional processing is required.

2. **Sorting the Coordinates**: The algorithm processes $n+1$ coordinates (including the origin). For simplicity, we denote the number of coordinates as $O(n)$. Sorting these coordinates by their $x$-coordinate and then by $y$-coordinate can be done efficiently using a comparison-based sorting algorithm like **Quick Sort**. This operation has a time complexity $O(n \log n)$.

3. **Generating the Path**: After sorting, the algorithm iterates through the coordinates to construct the path. For each pair of consecutive points $(x_i, y_i)$ and $(x_{i+1}, y_{i+1})$:
     - **Validity Check**: The comparisons $x_{i+1} \geq x_i$ and $y_{i+1} \geq y_i$ take $O(1)$.
     - **Calculate Steps**: Compute the horizontal and vertical moves required to reach the next coordinate:
         - Right moves (*R*): $x_{\text{diff}} = x_{i+1} - x_i$;
         - Up moves (*U*): $y_{\text{diff}} = y_{i+1} - y_i$;
         - Append these moves to the path string.
     - **Time Complexity Per Iteration**:
       - Each iteration executes two differences in $O(1)$ and appends at most $200$ moves ($100$ right and $100$ up) due to coordinate bounds ($0 \leq x_i, y_i \leq 100$), taking in totally $O(200) = O(1)$.
     - **Total Time Complexity for Path Generation**:
       - With $n$ iterations, the total time complexity is: $O(n \cdot 1) = O(n)$.

4. **Collecting Results**: Storing the result (either "YES" with the path or "NO") is a constant-time operation with respect to $n$, though it depends on the bounded length of the path. Given the constraints, this operation takes $O(1)$ time.

#### Total Time Complexity per Test Case
Adding the complexities from all steps, we have:
\begin{equation*}
\text{Time Complexity per Test Case} = O(1) + O(n \log n) + O(n) + O(1) = O(n \log n)
\end{equation*}
The **dominant steps** are:
- **Sorting the Coordinates**: $O(n \log n)$
- **Generating the Path**: $O(n)$

Thus, the overall time complexity is determined by the ***sorting step***,  resulting in a time complexity of $O(n \log n)$ for each test case.

#### Total Time Complexity Across All Test Cases
The algorithm processes $0 \leq t \leq 10$ test cases. Since $t$ is a small constant, the total time complexity is:
\begin{equation*}
\text{Total Time Complexity} = O(t \cdot n \log n) = O(10 \cdot n \log n) = O(n \log n)
\end{equation*}

### 4. Ask an LLM tool (such as ChatGPT, Claude AI, Gemini, Perplexity, etc.) to evaluate the time complexity of your code using Big O notation. Is the assessment accurate? If it differs from your previous analysis, which would be correct? Please explain your reasoning.

The LLM's assessment of the time complexity aligns with our analysis, concluding that the overall time complexity for each test case is dominated by the **sorting step**, which has a complexity of $O(n \log n)$. Extending this to all test cases, the total time complexity is $O(t \cdot n \log n)$. Given that the number of test cases satisfies $1 \leq t \leq 10$, this simplifies to $O(n \log n)$, which also coincides with our analysis. Other steps, including the origin point $(O(1))$, result collection $(O(1))$ and in particular path generation $(O(n))$, are correctly identified as secondary contributors to the computational cost.

Both analyses agree on the following key aspects:
1. **Sorting the Coordinates**: Both analyses identify sorting as the dominant operation, requiring $O(n \log n)$ due to the use of a comparison-based sorting algorithm like **Quick Sort** or **Merge Sort**, as noted by *ChatGPT*;
2. **Path Generation**: Both analyses agree that iterating through $n$ coordinates and generating the path contributes $O(n)$ to the time complexity;
3. **Overall Complexity**: Both analyses calculate the **time complexity per test case** as:

     \begin{equation*}
     O(1) + O(n \log n) + O(n) + O(1) = O(n \log n)
     \end{equation*}
     
     and the **total time complexity across all test cases** as:

     \begin{equation*}
     O(t \cdot n \log n) = O(10 \cdot n \log n) = O(n \log n)
     \end{equation*}

While *ChatGPT*'s analysis reaches the same conclusion regarding the overall complexity, its evaluation is less accurate in certain details. Our analysis explicitly explains why operations in the inner loop of path generation take constant time for each iteration. These involve bounded arithmetic and appending a limited number of characters to the path, which do not scale with $n$. The LLM does not delve into why these operations are constant-time, which is crucial for a more rigorous complexity analysis.

In conclusion, both analyses correctly determine the overall complexity, but our analysis is more detailed in justifying constant-time operations within the path generation step. This additional depth enhances clarity and ensures that the complexity assessment is fully justified.

### 5. Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not.

The greedy algorithm operates as follows: at each step, the robot moves to the nearest package based on the Manhattan distance. Formally, starting at position $(x_i, y_i)$, the robot selects the next package located at $(x_j, y_j)$ such that:

\begin{align*} j = \arg\min_{j \in S} \left( |x_i - x_j| + |y_i - y_j| \right), \end{align*}

where $S$ is the set of unvisited package locations. The robot then moves to $(x_j, y_j)$, removes $(x_j, y_j)$ from $S$, and repeats this process until $S$ is empty. 
This process repeats until all packages have been collected. We aim to analyze whether this approach always guarantees the shortest possible path.

To test the greedy algorithm, consider the following setup. The robot starts at position $(0,0)$, and the packages are located at: $\{(0,6)$, $(1,2)$, $(4,3)$, $(6,0)$, $(6,2)\}$. Let us compute the path that the greedy algorithm would take.

- **Initial Setup**
    - Starting position: $(0,0)$
    - Set of remaining packages, $S$: $\{(0,6), (1,2), (4,3), (6,0), (6,2)\}$

- **Iteration 1**: The robot begins at $(0,0)$. Let us calculate the distances to each package in $S$:
    - Distance to $(0,6)$: $|0 - 0| + |0 - 6| = 6$
    - Distance to $(1,2)$: $|0 - 1| + |0 - 2| = 3$
    - Distance to $(4,3)$: $|0 - 4| + |0 - 3| = 7$
    - Distance to $(6,0)$: $|0 - 6| + |0 - 0| = 6$
    - Distance to $(6,2)$: $|0 - 6| + |0 - 2| = 8$

    The nearest package is at $(1,2)$, with a distance of $3$. The robot moves to $(1,2)$.
    - New position: $(1,2)$
    - Updated set of remaining packages, $S$: $\{(0,6), (4,3), (6,0), (6,2)\}$
    - Total distance so far: $3$

- **Iteration 2**: From $(1,2)$, let us calculate the distances to each package in $S$:
    - Distance to $(0,6)$: $|1 - 0| + |2 - 6| = 5$
    - Distance to $(4,3)$: $|1 - 4| + |2 - 3| = 4$
    - Distance to $(6,0)$: $|1 - 6| + |2 - 0| = 7$
    - Distance to $(6,2)$: $|1 - 6| + |2 - 2| = 5$

    The nearest package is at $(4,3)$, with a distance of $4$. The robot moves to $(4,3)$.
    - New position: $(4,3)$
    - Updated set of remaining packages, $S$: $\{(0,6), (6,0), (6,2)\}$
    - Total distance so far: $3 + 4 = 7$

- **Iteration 3**: From $(4,3)$, let us calculate the distances to each package in $S$:
    - Distance to $(0,6)$: $|4 - 0| + |3 - 6| = 7$
    - Distance to $(6,0)$: $|4 - 6| + |3 - 0| = 5$
    - Distance to $(6,2)$: $|4 - 6| + |3 - 2| = 3$

    The nearest package is at $(6,2)$, with a distance of $3$. The robot moves to $(6,2)$.
    - New position: $(6,2)$
    - Updated set of remaining packages, $S$: $\{(0,6), (6,0)\}$
    - Total distance so far: $7 + 3 = 10$

- **Iteration 4**: From $(6,2)$, let us calculate the distances to each package in $S$:
    - Distance to $(0,6)$: $|6 - 0| + |2 - 6| = 10$
    - Distance to $(6,0)$: $|6 - 6| + |2 - 0| = 2$

    The nearest package is at $(6,0)$, with a distance of $2$. The robot moves to $(6,0)$.
    - New position: $(6,0)$
    - Updated set of remaining packages, $S$: $\{(0,6)\}$
    - Total distance so far: $10 + 2 = 12$

- **Iteration 5**: From $(6,0)$, there is only one package remaining in $S$:
    - Distance to $(0,6)$: $|6 - 0| + |0 - 6| = 12$
    
    The robot moves to $(0,6)$, collecting the final package.
    - New position: $(0,6)$
    - Updated set of remaining packages, $S$: $\emptyset$
    - Total distance so far: $12 + 12 = 24$

The total distance traveled by the robot using the greedy algorithm is:
\begin{equation*}
3 + 4 + 3 + 2 + 12 = 24.
\end{equation*}

This counterexample shows that although the greedy algorithm selects the nearest package at each step, it does not always minimize the total distance. By exhaustively comparing alternative sequences, a globally optimized path can be computed, which results in a shorter total distance, as shown in subsequent calculations. One such optimal path is as follows: the robot starts at $(0,0)$ and moves first to $(1,2)$, with a distance of $3$. From $(1,2)$, it moves to $(0,6)$, traveling $5$ units. Next, it moves to $(4,3)$, traveling $7$ units. From $(4,3)$, the robot moves to $(6,2)$, with a distance of $3$, and finally to $(6,0)$, with a distance of $2$. The total distance for this path is:
\begin{equation*}
3 + 5 + 7 + 3 + 2 = 20.
\end{equation*}

Interestingly, there is another path that also achieves the same optimal total distance of $20$. In this sequence, the robot starts at $(0,0)$ and moves directly to $(0,6)$, traveling $6$ units. From $(0,6)$, it moves to $(1,2)$, traveling $5$ units. Then, it moves to $(4,3)$, traveling $4$ units. From $(4,3)$, it moves to $(6,2)$, traveling $3$ units, and finally to $(6,0)$, traveling $2$ units. Again, the total distance is:
\begin{equation*}
6 + 5 + 4 + 3 + 2 = 20.
\end{equation*}

These calculations show that the greedy algorithm, while simple and intuitive, fails to minimize the total distance in this example. It results in a total distance of $24$, whereas the optimal path achieves only $20$. This counterexample illustrates that the greedy algorithm is not guaranteed to be optimal, as it focuses on local minimization rather than considering the global structure of the problem. An optimal algorithm, instead, should evaluate the total length of the remaining path rather than solely the distance to the next package.